#Instalación de Librerías y Módulos para el Notebook

In [ ]:
!pip install -q unidecode
!pip install -q easynmt
!pip install -q --upgrade gspread
!pip install -q gspread-formatting
!pip install -q google_spreadsheet
!pip install -q google-auth-oauthlib

     |████████████████████████████████| 235 kB 15.4 MB/s 
     |████████████████████████████████| 3.8 MB 11.2 MB/s 
     |████████████████████████████████| 1.2 MB 53.0 MB/s 
     |████████████████████████████████| 68 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 895 kB 56.4 MB/s 
     |████████████████████████████████| 6.5 MB 43.9 MB/s 
     |████████████████████████████████| 67 kB 3.3 MB/s 


#Montado de Sistema de Archivos de Google Drive

In [ ]:
from google.colab import drive

#NO MODIFICAR ESTO POR NINGUN MOTIVO
drive_root = "/content/drive/"

#Se mont el sistema de archivos de Drive
def montar_Drive():
  drive.mount("/content/drive/", force_remount=True)

montar_Drive()
drive_root = "/content/drive/Shareddrives/"

#Esta son las rutas con las que se van a estar trabajando en el proyecto
project_root = drive_root + '(50 PROJ-CORP) DATA ANALYTICS/Proyectos/Investigación/Hosteltur/Scripts/'
BW_dir = project_root + "Archivos_BW/"
BW_raw = BW_dir + "Originales/"
BW_sampled = BW_dir + "Muestreados/"
BW_cleaned = BW_dir + "Traducidos-Depurados/"
matrixes_dir = project_root + "Matrices_Resultantes/"
scripts_dir = project_root + "Scripts_Colab/"
csvs_dir = project_root + "Archivos_Auxiliares(CSV)/"

Mounted at /content/drive/


#Importación de Módulos y Librerías para el Notebook

In [ ]:
import pandas as pd
import numpy as np
import glob
import re
from tqdm.notebook import tqdm_notebook
from ipywidgets import IntProgress
from unidecode import unidecode
from easynmt import EasyNMT
from IPython.display import display, HTML
from tqdm import tqdm
import os
import sys
import zipfile
import shutil
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import gspread_formatting as gsf 
import time
from gspread_formatting.models import Borders
from google.cloud import storage
import spacy

#Constantes y Variables "Globales"

In [ ]:
SIMPLE_BW_COLUMNS = ["Full Text", "Country Code", "Location Name", "Language", "Page Type", "Date", "Sentiment", "Gender", "Reach (new)", "Url"] #Columnas a conservar de archivos BW originales
SAMPLE_MAX_SIZE = 2000000 #Tamaño máximo de los archivos muestra a manejar
ARCHIVO_DESTINOS_CLASIFICACION = csvs_dir + ""
MATRIZ_RESUMEN = "Matriz_Resumen"
MATRIZ_DESGLOSE = "Copia de Matriz_Desglose"
DICCIONARIO_MATRIZ_SIMPLE = "Diccionario_PalabrasBusqueda"
MATRIX_START_ROW = 8
SLEEP_TIME = 0.35
#Formato de porcentajes para matrices en spreadsheet
cell_decimal_format = gsf.cellFormat(
    numberFormat = gsf.numberFormat("NUMBER", pattern = "##.###%") 
  )
cell_integer_format = gsf.cellFormat(
    numberFormat = gsf.numberFormat("NUMBER", pattern = "##")
)
border_style = gsf.Border("SOLID", {"red": 0, "green": 0,"blue": 0, "alpha": 1}, 1, None)
cell_margins = gsf.cellFormat(
    borders = gsf.Borders(border_style, border_style, border_style, border_style)
)

#Samplear los archivos de Brandwatch

In [ ]:
montar_Drive()
tqdm_notebook.pandas()

for f in glob.glob(BW_raw +'*'):
  m = re.search('(\w+)\.csv\.zip$', f) #Busca archivos de BW
  co = "ESP"
  output_file = m[1] + ".csv" #Indica nombre de archivo re_sampleado
  print('Procesando... {}'.format(m.group(1)))

  if os.path.isfile(output_file): #En caso de que el archivo original de BW ya haya sido muestreado
      print( '...el fichero muestreado ya existe ({})'.format(output_file))
      exit
  else:
      do_sample = True
      
      no_tuits = SAMPLE_MAX_SIZE #Definicion de size de la muestra
      
      print('A {} le corresponden {:,} tuits'.format(m.group(1),no_tuits))

      df = pd.read_csv(f, skiprows=6, usecols=SIMPLE_BW_COLUMNS)
      df = df[SIMPLE_BW_COLUMNS] #Se lee solamente las columnas necesarias del BW

      print('Hay: {:,} tuits'.format(df.shape[0])) 
      if df.shape[0] <= no_tuits: #El size del archivo es menor al necesario para realizar muestra
          print('Hay menos tuits de los que debería: no samplearé')
          do_sample = False

      try:
        if do_sample:
            df = pd.concat([df.sample(n=no_tuits, random_state=1)]) #Hace una muestra completamente aleatoria del archivo, con el size previamente delimitado
            print('Ahora son...')
            print('Quedan {:,} tuits'.format(df.shape[0]))
            print('Escribiendo... {}'.format(output_file))
            #Escritura del nuevo archvo
        df.to_csv(output_file, index=False)
        with zipfile.ZipFile(m[1]+".csv.zip", "w", zipfile.ZIP_DEFLATED) as archive:
          archive.write(output_file)
          archive.close()
          shutil.move(m[1]+".csv.zip", BW_sampled)
          os.remove(output_file)
      except:
        print( '...el fichero muestreado ya existe ({})'.format(output_file))
        os.remove(output_file)
        os.remove(m[1]+".csv.zip")


Mounted at /content/drive/
Procesando... nacionales
A nacionales le corresponden 2,000,000 tuits
Hay: 1,051,786 tuits
Hay menos tuits de los que debería: no samplearé
...el fichero muestreado ya existe (nacionales.csv)
Procesando... internacionales
A internacionales le corresponden 2,000,000 tuits
Hay: 515,075 tuits
Hay menos tuits de los que debería: no samplearé
...el fichero muestreado ya existe (internacionales.csv)
Procesando... Hosteltur_2019EmpresasAcotada
A Hosteltur_2019EmpresasAcotada le corresponden 2,000,000 tuits
Hay: 37,356 tuits
Hay menos tuits de los que debería: no samplearé
...el fichero muestreado ya existe (Hosteltur_2019EmpresasAcotada.csv)
Procesando... Hosteltur_2020EmpresasAcotada
A Hosteltur_2020EmpresasAcotada le corresponden 2,000,000 tuits
Hay: 43,878 tuits
Hay menos tuits de los que debería: no samplearé
...el fichero muestreado ya existe (Hosteltur_2020EmpresasAcotada.csv)
Procesando... Hosteltur_2021EmpresasAcotada
A Hosteltur_2021EmpresasAcotada le corre

#Traducción y Limpieza de la Información

In [ ]:
#Eliminación de acentos
def quitarAcentos(full_text):
  sin_acentos = (
      ("á","a"),
      ("é","e"),
      ("í","i"),
      ("ó","o"),
      ("ú","u")
  )
  for acento, sin_acento in sin_acentos:
    full_text = full_text.replace(acento, sin_acento)
  return full_text

#Eliminación de handles, links, hashtags y espacios vacios en el texto
def simple_tweet(full_text):
    # handles
    out = re.sub(r'@\w+', '', str(full_text))
    # urls
    out = re.sub(r'https?://[\w&#\?\.\/]+', '', out)
    # hashtags
    out = re.sub(r'#[\w_\-]+', '', out)
    # a minusculas
    out = out.lower()
    # acentos
    out = quitarAcentos(out)
    # espacios en blanco
    out = out.strip()
    return out

montar_Drive()
tqdm_notebook.pandas()

model = None

for f in glob.glob(BW_sampled+'*'):
  m = re.search('(\w+)\.csv\.zip$', f)
  co = "ESP"
  output_file = m[1] + ".csv" #Indica nombre de archivo re_sampleado

  if os.path.isfile(output_file):
      print( '...el fichero de traducción ya existe ({})'.format(output_file) )
  else:
    if model is None: #Carga del modelo para la traducción
        print( 'Cargando el modelo de traducción...' )
        model = EasyNMT('m2m_100_1.2B')
        print( '...cargado\n' )

    TARGET_LANG = "es"
    print( 'Target lang: {}'.format(TARGET_LANG) )
    print(f)

    df = pd.read_csv(f)
    print('Limpiando tweets... {:,}'.format(df.shape[0]))
    df['Full Text'] = df.progress_apply( lambda x: simple_tweet(x['Full Text']), axis=1 ) #Se aplica la depuracion de cada uno de los tweets

    #Traduccion de los tweets
    print( 'Idiomas en el dataset...' )
    display( df.groupby('Language')['Language'].count().sort_values(ascending=False).iloc[0:8] )

    for i, row in tqdm(df[(df.Language!=TARGET_LANG)&(df.Language!='undefined')].iterrows(), total=df[(df.Language!=TARGET_LANG)&(df.Language!='undefined')].shape[0]):
        try:
            df.loc[i, 'Full Text'] = model.translate( row['Full Text'], 
                                                            source_lang=row['Language'], 
                                                            target_lang=TARGET_LANG )
        except:
            pass

    #Escritura de Archivos
    print('Escribiendo... {}'.format(output_file))
    try:
      df.to_csv(output_file, index=False)
      with zipfile.ZipFile(m[1]+".csv.zip", mode="w") as archive:
        archive.write(output_file)
      shutil.move(m[1]+".csv.zip", BW_cleaned)
      os.remove(output_file)
    except:
        print( '...el fichero muestreado ya existe ({})'.format(output_file))
        os.remove(output_file)
        os.remove(m[1]+".csv.zip")

Mounted at /content/drive/
Cargando el modelo de traducción...


89.9kB [00:00, 11.2MB/s]                   


Downloading:   0%|          | 0.00/909 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.62G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/271 [00:00<?, ?B/s]

...

  0%|          | 0/512811 [00:00<?, ?it/s]

Idiomas en el dataset...


Language
es    512811
Name: Language, dtype: int64

0it [00:00, ?it/s]

Escribiendo... Hosteltur_DestinosInternacionalesSinPolitica.csv


#Funciones Diversas

In [ ]:
#Lecuta de archivo con palabras de busqueda
def readSimpleSearchWords(file_ID):
  auth.authenticate_user()
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  document = gc.open_by_key(file_ID)
  words = document.worksheet(DICCIONARIO_MATRIZ_SIMPLE).get_all_values()[1:] 
  df = pd.DataFrame(words, columns=document.worksheet(DICCIONARIO_MATRIZ_SIMPLE).get_all_values()[0])
  return [df, document]

#Funcion para formateo de Fecha para filtros de fechas
def formateoFecha(fecha):
  fecha = fecha[0:10]
  return fecha

def regularExpressionCreation(words_x, words_y):
  expression_x = expression_y = "("
  expressions = [expression_x, expression_y]
  for list_words, expression in zip ([words_x, words_y], range(0,2)):
    for word in list_words:
      if "*" in word and word != list_words[len(list_words)-1]:
        sub_expression = "(\\b"+ word[0:len(word)-1] + ")|"
      elif "*" in word and word == list_words[len(list_words)-1]:
        sub_expression = "(\\b" + word[0:len(word)-1] + ")"
      elif "*" not in word and word != list_words[len(list_words)-1]:
        sub_expression = "(\\b"+ word + "(\\b|\W{1,2}))|" 
      else:
        sub_expression = "(\\b" + word + "(\\b|\W{1,2}))"
      expressions[expression] = expressions[expression] + sub_expression
    expressions[expression] = expressions[expression] + ")"
  return expressions

#Matrices Sencillas

In [ ]:
#Creacion de diccionaro para busquedas
def elaborateSimpleDictonary(search_words):
  search_dictonary = dict()
  for index, row in search_words.iterrows():
    
    #Nombre_destino, Opciones_busqueda, Ubicacion, Tipo
    search_options = str(row['Query'])
    search_options = search_options.replace(", ", ",") #Manejo de comas 
    search_options = search_options.replace(u'\xa0', "") #Espacios en blanco al final de texto son eliminados
    search_options_list = list()
    
    separator_index = 0
    for char, index in zip(search_options, range(0, len(search_options))): #Separacion de palabras/opciones de busqueda por categoria/marca/destino/etc
      if char == ",":
        if separator_index != 0:
          search_options_list.append(search_options[separator_index+1:index].lower())
        else:
          search_options_list.append(search_options[separator_index:index].lower())
        separator_index = index
    if separator_index == 0:
     search_options_list.append(search_options.lower())
    else:
      search_options_list.append(search_options[separator_index+1:len(search_options)].lower())
    
    search_dictonary[row['Label']] = (search_options_list, row["Ubicacion"], row["Tipo"])
  return search_dictonary

In [ ]:
def simpleMatrix(file_path, second_file_path, file_ID, date_filter):
  auth.authenticate_user()
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  datalake = pd.read_csv(file_path) #Datalake Nacionales
  second_datalake = pd.read_csv(second_file_path) #Datalake Internacionales
  datalake = pd.concat([datalake, second_datalake]).drop_duplicates().reset_index(drop=True) #Union de ambos datalakes para elaboración de matriz
  datalake["Date"].apply(formateoFecha) 
  search_words, document = readSimpleSearchWords(file_ID[0]) #Carga de palabras y queries a buscar desde el spreadsheet
  search_dictonary = elaborateSimpleDictonary(search_words) #Elaboracion de diccionario
  
  for filee, date in zip(file_ID, date_filter):
    document = gc.open_by_key(filee)
    df = datalake.copy()
    df = (df.loc[(df["Date"] >= date[0]) & (df["Date"] < date[1])])
 
    datalake_size = len(df)
    datalake_mentions = datalake_percentage_mentions = men_mentions = women_mentions = other_mentions = 0
    detailed_matrix = []
    summary_matrix = {}

    for element in search_dictonary.keys():
      if search_dictonary[element][1] + " - " + search_dictonary[element][2] not in summary_matrix:
        #Esta lista se usará para ir acumulando las estadisticas por categoria para la matriz "Resumida"
        summary_matrix[search_dictonary[element][1] + " - " + search_dictonary[element][2]] = [0,0,0,0,0,0,0,0] #Menciones, m. hombres, m. mujeres, twitter, news, otras fuentes, prom positivo, prom negativo
      
      #Se obtienen números absolutos sobre estadísticas para cada término/Ciudad/elemento a ser escritas en la matriz Desglosada
      df_element = df[df["Full Text"].str.contains("|".join(search_dictonary[element][0]), na=False)]
      men_mentions_element = len(df_element[(df_element["Gender"] == "male") & (df_element["Page Type"] == "twitter")])
      men_mentions += men_mentions_element
      summary_matrix[search_dictonary[element][1] + " - " + search_dictonary[element][2]][1] += men_mentions_element
      women_mentions_element = len(df_element[(df_element["Gender"] == "female") & (df_element["Page Type"] == "twitter")])
      women_mentions += women_mentions_element
      summary_matrix[search_dictonary[element][1] + " - " + search_dictonary[element][2]][2] += women_mentions_element
      other_mentions += len(df_element) - (men_mentions_element + women_mentions_element)
      datalake_mentions += len(df_element)
      positive_sentiment = len(df_element[df_element["Sentiment"] == "positive"])
      negative_sentiment = len(df_element[df_element["Sentiment"] == "negative"])

      if len(df_element) == 0: #Manejo de casos cuando el término/elemento reporta una cuenta de 0 elementos
        men_percentage = female_percentage = twitter_percentage = news_percentage = others_percentage = positive_percentage = negative_percentage = 0
      else:
        if men_mentions_element == 0 or women_mentions_element == 0: #Manejo de estadisticas de genero
          if men_mentions_element == 0:
              men_percentage = 0
              female_percentage = 1
          else:
              men_percentage = 1
              female_percentage = 0
        else:
            men_percentage = round(((men_mentions_element) / (men_mentions_element + women_mentions_element)), 4)
            female_percentage = round(((women_mentions_element) / (men_mentions_element + women_mentions_element)), 4)
        twitter_percentage = round((len(df_element[df_element["Page Type"] == "twitter"]) / len(df_element)), 4)
        news_percentage = round((len(df_element[df_element["Page Type"] == "news"]) / len(df_element)), 4)
        others_percentage = round((1 - (twitter_percentage + news_percentage)), 4)
        positive_percentage = round((positive_sentiment / len(df_element)), 4)
        negative_percentage = round((negative_sentiment / len(df_element)), 4)

      values = [len(df_element[df_element["Page Type"] == "twitter"]), len(df_element[df_element["Page Type"] == "news"]),  
                len(df_element) - (len(df_element[df_element["Page Type"] == "twitter"]) + len(df_element[df_element["Page Type"] == "news"])), positive_percentage, negative_percentage]
      for value, index in zip(values, range(3,8)):
          summary_matrix[search_dictonary[element][1] + " - " + search_dictonary[element][2]][index] += value
        
      #Elemento buscado, Categoria, N. menciones, %menciones en categoria, %menciones hombres, %menciones mujeres, %twitter, %news, %otras fuentes, %S+, %S-, NPS (Escritura en el archivo de Sheets)
      detailed_matrix.append([element, search_dictonary[element][1] + " - " + search_dictonary[element][2], len(df_element), 0, men_percentage, female_percentage, 
                              twitter_percentage, news_percentage, others_percentage, positive_percentage, negative_percentage, round((positive_percentage - negative_percentage) * 100, 4)])
      summary_matrix[search_dictonary[element][1] + " - " + search_dictonary[element][2]][0] += len(df_element)

    for element in detailed_matrix:
      if summary_matrix[element[1]][0] > 0:
        element[3] = round((element[2] / summary_matrix[element[1]][0]), 4)

    datalake_percentage_mentions = round(((datalake_mentions / datalake_size)), 4)
    twitter_mentions = men_mentions + women_mentions

    tabs = [MATRIZ_RESUMEN, MATRIZ_DESGLOSE]
    #Escritura de estadisticas del datalake en general
    for tab in tabs:
      tab_to_write = document.worksheet(tab)
      tab_to_write.update("B2", [[datalake_mentions], [datalake_size], [datalake_percentage_mentions]])
      gsf.format_cell_range(tab_to_write, "B4", cell_decimal_format)

    matrix_start_row = MATRIX_START_ROW
    #Escritura de cada una de las filas de la matriz "Desglosada" (a detalle y por cada elemento siendo buscado)
    for element in detailed_matrix:
      index = "A"+str(matrix_start_row)
      second_index = "F"+str(matrix_start_row)
      #Se dejan las columnas de de "% Menciones respecto al total" y NPS fuera de la automatizacion del script
      tab_to_write.update(index, [[element[0], element[1], element[2], element[3]]]) 
      tab_to_write.update(second_index, [[element[4], element[5], element[6], element[7], element[8], element[9], element[10]]])
      gsf.format_cell_range(tab_to_write, "D"+str(matrix_start_row)+ ":" + "L"+str(matrix_start_row), cell_decimal_format)
      gsf.format_cell_range(tab_to_write, index + ":" + "M"+str(matrix_start_row), cell_margins)
      matrix_start_row +=1
      time.sleep(SLEEP_TIME)

    matrix_start_row = MATRIX_START_ROW
    tab_to_write = document.worksheet(MATRIZ_RESUMEN)
    for element in summary_matrix: #Obtencion de porcentajes de diferentes estadisticas, para cada una de las categorías
      index = "A"+str(matrix_start_row)
      second_index = "D"+str(matrix_start_row)
      category_elements = sum(row.count(element) for row in detailed_matrix)
      
      category_percentage = round((summary_matrix[element][0] / datalake_mentions),4)
      
      if summary_matrix[element][0] == 0:
        twitter_percentage = news_percentage = others_percentage = 0
      else:
        twitter_percentage = round((summary_matrix[element][3] / summary_matrix[element][0]),4)
        news_percentage = round((summary_matrix[element][4] / summary_matrix[element][0]),4)
        others_percentage = round((summary_matrix[element][5] / summary_matrix[element][0]),4)
      
      if category_elements == 0:
        positive_percentage = negative_percentage = 0
      else:
        positive_percentage = round((summary_matrix[element][6] / category_elements),4)
        negative_percentage = round((summary_matrix[element][7] / category_elements),4)
      Nps = round((positive_percentage - negative_percentage) * 100, 4)

      if summary_matrix[element][1] == 0 or summary_matrix[element][2] == 0: #Manejo de estadisticas de genero
        if summary_matrix[element][1] == 0 and summary_matrix[element][2]:
            men_percentage = 0
            female_percentage = 0
        elif summary_matrix[element][1] == 0:
            men_percentage = 0
            female_percentage = 1
        else:
            men_percentage = 1
            female_percentage = 0
      else:
        men_percentage = round(((summary_matrix[element][1]) / (summary_matrix[element][1] + summary_matrix[element][2])), 4)
        female_percentage = round(((summary_matrix[element][2]) / (summary_matrix[element][1] + summary_matrix[element][2])), 4)

      #Escritura de cada una de las filas de la matriz por categorías (Condensada)
      tab_to_write.update(index, [[element, summary_matrix[element][0]]])
      tab_to_write.update(second_index, [[men_percentage, female_percentage, twitter_percentage, news_percentage, others_percentage, positive_percentage, negative_percentage]])
      gsf.format_cell_range(tab_to_write, "C"+str(matrix_start_row)+ ":" + "J"+str(matrix_start_row), cell_decimal_format)
      gsf.format_cell_range(tab_to_write, index + ":" + "K"+str(matrix_start_row), cell_margins)
      matrix_start_row +=1
      time.sleep(SLEEP_TIME)
    print(f'El archivo {document.title} ha terminado de escribir sus matrices')
    del df

#Matrices Complejas

In [ ]:
 #Funcion auxiliar para creacion de diccionario
def complexDictonaryAux(search_options, search_options_list, search_dictonary, row):
    separator_index = 0
    for char, index in zip(search_options, range(0, len(search_options))): #Separacion de palabras/opciones de busqueda por categoria/marca/destino/etc
      if char == ",":
        if separator_index != 0:
          search_options_list.append(search_options[separator_index+1:index].lower())
        else:
          search_options_list.append(search_options[separator_index:index].lower())
        separator_index = index
    if separator_index == 0:
     search_options_list.append(search_options.lower())
    else:
      search_options_list.append(search_options[separator_index+1:len(search_options)].lower())
    if search_options != "":
      search_dictonary[row] = (search_options_list)

#Creacion de diccionaro para busquedas
def elaborateComplexDictonary(search_words):
  search_dictonary_x, search_dictonary_y = dict(), dict()
  for index, row in search_words.iterrows():
    search_options_x, search_options_y = str(row["Query (Eje x)"]), str(row["Query (Eje y)"])
    search_options_x, search_options_y = search_options_x.replace(", ", ","), search_options_y.replace(", ", ",")
    search_options_x, search_options_y = search_options_x.replace(u'\xa0', ""), search_options_y.replace(u'\xa0', "")
    search_options_list_x, search_options_list_y = list(),list()

    complexDictonaryAux(search_options_x, search_options_list_x, search_dictonary_x, row["Label (Eje x)"])
    complexDictonaryAux(search_options_y, search_options_list_y, search_dictonary_y, row["Label (Eje y)"])
    
  return [search_dictonary_x, search_dictonary_y]

In [ ]:
def complexMatrix(file_path, second_file_path, file_ID, near_and, date_filter):
  auth.authenticate_user()
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  datalake = pd.read_csv(file_path) #Datalake Nacionales
  second_datalake = pd.read_csv(second_file_path) #Datalake Internacionales
  #datalake = pd.concat([datalake, second_datalake]).drop_duplicates().reset_index(drop=True) #Union de ambos datalakes para elaboración de matriz
  datalake["Date"].apply(formateoFecha) 
  search_words, document = readSimpleSearchWords(file_ID) #Carga de palabras y queries a buscar desde el spreadsheet
  search_dictonary_x, search_dictonary_y = elaborateComplexDictonary(search_words) #Elaboracion de diccionarios (uno para eje X y otro para eje Y)
  datalake = (datalake.loc[(datalake["Date"] >= date_filter[0]) & (datalake["Date"] < date_filter[1])])

  datalake_mentions = datalake_percentage_mentions = men_mentions = women_mentions = other_mentions = 0
  datalake_size = len(datalake)

  shorten_dataframes_x = []
  shorten_dataframes_y = []

  document = gc.open_by_key(file_ID)
  tab_to_write_one = document.worksheet(MATRIZ_RESUMEN)
  tab_to_write_two = document.worksheet(MATRIZ_DESGLOSE)

  for element_x in search_dictonary_x.keys():
    df = datalake[datalake["Full Text"].str.contains('|'.join(search_dictonary_x[element_x]),na=False)]
    shorten_dataframes_x.append(df)

  for element_y in search_dictonary_y.keys():
    df = datalake[datalake["Full Text"].str.contains('|'.join(search_dictonary_y[element_y]), na=False)]
    shorten_dataframes_y.append(df)

  matrix_start_row = MATRIX_START_ROW
  matrix_start_column = 2
  detailed_start_row = MATRIX_START_ROW

  for element_x, element_x_key in zip(shorten_dataframes_x, search_dictonary_x.keys()):
    index = "A"+str(matrix_start_row)
    tab_to_write_one.update(index, element_x_key)
    matrix_start_column = 2
    category_mentions = 0
    detailed_matrix_rows = []
    for element_y, element_y_key in zip(shorten_dataframes_y, search_dictonary_y.keys()):
      if element_x_key == list(search_dictonary_x.keys())[0]:
        tab_to_write_one.update_cell(7, matrix_start_column, str(element_y_key))
       
      detailed_matrtix_key = str(element_x_key) + " - " + str(element_y_key)
      expression_x, expression_y = regularExpressionCreation(search_dictonary_x[element_x_key], search_dictonary_y[element_y_key])
      expression_halfOne = expression_halfTwo = ""
      df = pd.concat([element_x, element_y]).drop_duplicates().reset_index(drop=True)
      filtered_df = pd.DataFrame(columns=SIMPLE_BW_COLUMNS)

      if near_and.lower() == "near":
        df_halftwo = df[df["Full Text"].str.contains(expression_y + r"(?:(\s*\w*\W*){"+ str(0)+r","+str(4)+r"}?)" + expression_x)==True]
        df_halfone = df[df["Full Text"].str.contains(expression_x + r"(?:(\s*\w*\W*){"+ str(0)+r","+str(4)+r"}?)" + expression_y)==True]
        if len(df_halfone) == 0 and len(df_halftwo) == 0:
          pass
        elif len(df_halfone) == 0:
          filtered_df = pd.concat([filtered_df,df_halftwo]).drop_duplicates()
        elif len(df_halftwo) == 0:
          filtered_df = pd.concat([filtered_df, df_halfone]).drop_duplicates()
        else:
          filtered_df = pd.concat([filtered_df,df_halfone, df_halftwo]).drop_duplicates()
        df = filtered_df
      else:
        # df_element = df[df["Full Text"].str.contains("|".join(search_dictonary[element][0]), na=False)] PARA LOS PAISES USAR ESTA
        filtered_df = df[(df["Full Text"].str.contains(expression_x)) & (df["Full Text"].str.contains(expression_y))]
        df = filtered_df

      tab_to_write_one.update_cell(matrix_start_row, matrix_start_column, round((len(df)*1.30)))
      time.sleep(SLEEP_TIME)
      matrix_start_column += 1
      datalake_mentions += round((len(df)*1.30))

      mention_number = round((len(df)*1.30))
      category_mentions += round((len(df)*1.30))
      
      men_mentions_element = len(filtered_df[(filtered_df["Gender"] == "male") & (filtered_df["Page Type"] == "twitter")])
      women_mentions_element = len(filtered_df[(filtered_df["Gender"] == "female") & (filtered_df["Page Type"] == "twitter")])
      other_mentions += len(filtered_df) - (men_mentions_element + women_mentions_element)
      positive_sentiment = len(filtered_df[filtered_df["Sentiment"] == "positive"])
      negative_sentiment = len(filtered_df[filtered_df["Sentiment"] == "negative"])

      if len(filtered_df) == 0: #Manejo de casos cuando el término/elemento reporta una cuenta de 0 elementos
        men_percentage = female_percentage = twitter_percentage = news_percentage = others_percentage = positive_percentage = negative_percentage = 0
      else:
        if men_mentions_element == 0 or women_mentions_element == 0: #Manejo de estadisticas de genero
          if men_mentions_element == 0:
            men_percentage = 0
            female_percentage = 1
          else:
            men_percentage = 1
            female_percentage = 0
        else:
            men_percentage = round(((men_mentions_element) / (men_mentions_element + women_mentions_element)), 4)
            female_percentage = round(((women_mentions_element) / (men_mentions_element + women_mentions_element)), 4)

        twitter_percentage = round((len(filtered_df[filtered_df["Page Type"] == "twitter"]) / len(filtered_df)), 4)
        news_percentage = round((len(filtered_df[filtered_df["Page Type"] == "news"]) / len(filtered_df)), 4)
        others_percentage = round((1 - (twitter_percentage + news_percentage)), 4)
        positive_percentage = round((positive_sentiment / len(filtered_df)), 4)
        negative_percentage = round((negative_sentiment / len(filtered_df)), 4)
    
      detailed_matrix_rows.append([detailed_matrtix_key, mention_number, 0, men_percentage, female_percentage, twitter_percentage, news_percentage, others_percentage, positive_percentage, negative_percentage])

    for row in detailed_matrix_rows:
      index = "A"+str(detailed_start_row)
      second_index = "E"+str(detailed_start_row) 
      if category_mentions == 0 or row[1] == 0:
        pass
      else:
        row[2] = round((row[1] / category_mentions), 4)
      tab_to_write_two.update(index, [[row[0], row[1], row[2]]]) 
      tab_to_write_two.update("D"+str(detailed_start_row), ("=(B"+str(detailed_start_row)+"/$B$"+str(2)+")"), raw=False)
      tab_to_write_two.update(second_index, [[row[3], row[4], row[5], row[6], row[7], row[8], row[9]]])
      gsf.format_cell_range(tab_to_write_two, "C"+str(detailed_start_row)+ ":" + "K"+str(detailed_start_row), cell_decimal_format)
      gsf.format_cell_range(tab_to_write_two, index + ":" + "N"+str(detailed_start_row), cell_margins)
      tab_to_write_two.update("L"+str(detailed_start_row), ("=(J"+str(detailed_start_row)+"-K"+str(detailed_start_row)+")*100"), raw=False)
      if row[8] != 0 or row[9] != 0:
        gsf.format_cell_range(tab_to_write_two, "L"+str(detailed_start_row), cell_integer_format)
      if row[8] == 0:
        tab_to_write_two.update("M"+str(detailed_start_row), [[0]])
      else:
        tab_to_write_two.update("M"+str(detailed_start_row), ("=(B"+str(detailed_start_row)+"/(B"+str(detailed_start_row)+"*J"+str(detailed_start_row)+"))"), raw=False)
        gsf.format_cell_range(tab_to_write_two, "M"+str(detailed_start_row), cell_integer_format)
      if row[9] == 0:
        tab_to_write_two.update("N"+str(detailed_start_row), [[0]])
      else:
        tab_to_write_two.update("N"+str(detailed_start_row), ("=(B"+str(detailed_start_row)+"/(B"+str(detailed_start_row)+"*K"+str(detailed_start_row)+"))"), raw=False)
        gsf.format_cell_range(tab_to_write_two, "N"+str(detailed_start_row), cell_integer_format)
      time.sleep(SLEEP_TIME)
      detailed_start_row += 1  
    matrix_start_row += 1 
     
  datalake_percentage_mentions = round(((datalake_mentions / datalake_size)), 4)
  tab_to_write_one.update("B2", [[datalake_mentions], [datalake_size], [datalake_percentage_mentions]])
  gsf.format_cell_range(tab_to_write_one, "B4", cell_decimal_format)
  tab_to_write_two.update("B2", [[datalake_mentions], [datalake_size], [datalake_percentage_mentions]])
  gsf.format_cell_range(tab_to_write_one, "B4", cell_decimal_format)

# Ejecución de funciones para generar Matrices

In [ ]:
simpleMatrix(BW_cleaned+"Hosteltur_DestinosNacionalesSinPolitica.csv.zip", BW_cleaned+"Hosteltur_DestinosInternacionalesSinPolitica.csv.zip", 
             ["1v4kMvAHofeLMcF7Bgzs3nYbEfWKBJ98fyFWMdSaUbHs"]
             #["1K4zPAK_f6RfYPaaFTL_oAHLi8PMwq9Qx1fRWPn0j1Tc", "16BAkqqHpXx9r86b40WQ-DoEyhGrtVBc8AtQj32RBsJU", "1We2mH_JyZFP1CHu5d7ooc6l-swR1UTCz50w-a8xlbHM", "1fCidUucULKjeSDD5QV-3NBIufg6Wp8r52StU2DWqUDk"],
             [["2019-01-01", "2022-01-01"], ["2019-01-01", "2020-01-01"], ["2020-01-01", "2021-01-01"], ["2021-01-01", "2022-01-01"]]) #Pregunta 2

El archivo Pregunta2_Completa ha terminado de escribir sus matrices
El archivo Pregunta2_2019 ha terminado de escribir sus matrices
El archivo Pregunta2_2020 ha terminado de escribir sus matrices
El archivo Pregunta2_2021 ha terminado de escribir sus matrices


In [ ]:
complexMatrix(BW_cleaned+"2019_55.zip", BW_cleaned+"internacionales.csv.zip", "1_P5gJb8b0VyfCkPFTg-FfnzZ5RWIMjRbCNThafjM8B0", "NEAR", ["2019-01-01", "2020-01-01"]) 